# Comparison of Reference ET module calculations using MACA

In [50]:
import math
import numpy as np

import ee
import refet

import openet.refetgee.daily


ee.Initialize()

# test point for reducer
lat = 39.15
lon = -110.0583333

# test date
date = '2006-01-13'

#test maca scenario, model
scen = 'rcp45'
mod = 'bcc-csm1-1-m'

# random image from collection
maca_img = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')\
    .filterMetadata('scenario', 'equals', scen)\
    .filterMetadata('model', 'equals', mod)\
    .filterDate(date).first()

# Set the output crs and crsTransform to match the maca image
crs = maca_img.projection().getInfo()['crs']
geo = maca_img.projection().getInfo()['transform']


test_pnt = ee.Geometry.Point(lon, lat)

# gee_refet eto
gee_eto = openet.refetgee.daily.Daily.maca(maca_img).eto\
    .rename(['eto'])\
    .reduceRegion(ee.Reducer.mean(), test_pnt, crs=crs, crsTransform=geo)\
    .getInfo()['eto']

#latitude in the maca class is in radians (convert to degrees)
print('Maca Class Latitude: {}'.format(math.degrees(openet.refetgee.daily.Daily.maca(maca_img).lat\
    .rename(['lat'])\
    .reduceRegion(ee.Reducer.mean(), test_pnt, crs=crs, crsTransform=geo)\
    .getInfo()['lat'])))

# gee_refet etr
gee_etr = openet.refetgee.daily.Daily.maca(maca_img).etr\
    .rename(['etr'])\
    .reduceRegion(ee.Reducer.mean(), test_pnt, crs=crs, crsTransform=geo)\
    .getInfo()['etr']

# get point data for RefET calc/comparison
pt_data = maca_img.reduceRegion(ee.Reducer.mean(), test_pnt, crs=crs,
                                crsTransform=geo).getInfo()
# image doy
image_date = ee.Date(maca_img.get('system:time_start'))
doy = ee.Number(image_date.getRelative('day', 'year')).add(1).double().getInfo()

# elevation pt extraction (gridmet elevation asset)
elev_img = ee.Image('projects/earthengine-legacy/assets/'
                    'projects/climate-engine/gridmet/elevation')
elev_crs = elev_img.projection().getInfo()['crs']
elev_geo = elev_img.projection().getInfo()['transform']
elev_pt = elev_img.reduceRegion(ee.Reducer.mean(), test_pnt, crs=elev_crs,
                                crsTransform=elev_geo).getInfo()

# gridcell lat from elevation asset
lat_img =ee.Image('projects/earthengine-legacy/assets/'
                  'projects/climate-engine/gridmet/elevation') \
    .multiply(0).add(ee.Image.pixelLonLat().select('latitude'))
cell_lata_elev = lat_img.reduceRegion(ee.Reducer.mean(), test_pnt, crs=elev_crs,
                                crsTransform=elev_geo).getInfo()['b1']
print('Elevation Image Latitude: {}'.format(cell_lata_elev))

# grid cell lat from maca img
lat_img = ee.Image(maca_img).multiply(0).add(ee.Image.pixelLonLat().select('latitude'))
# print(lat_img)
# choose random band (huss)
cell_lat = lat_img.reduceRegion(ee.Reducer.mean(), test_pnt, crs=crs,
                                crsTransform=geo).getInfo()['huss']
print('Input Image Latitude: {}'.format(cell_lat))

# air pressure from gridmet elevation using refet module
pair_kpa = refet.calcs._air_pressure(round(elev_pt['b1'], 2), method='asce')
# actual vapor pressure (kg/kg) using refet module
ea_kpa = refet.calcs._actual_vapor_pressure(pt_data['huss'], pair_kpa)

# resultant wind speed
windspeed = np.sqrt(pt_data['uas']**2 + pt_data['vas']**2)
windspeed_2m = refet.calcs._wind_height_adjust(
            windspeed, 10)

# refet etr calc
etr = refet.Daily(
    tmin=pt_data['tasmin'], tmax=pt_data['tasmax'], ea=ea_kpa,
    rs=pt_data['rsds'],
    uz=windspeed, zw=10, elev=round(elev_pt['b1'], 2),
    lat=cell_lat, doy=doy, method='asce',
    input_units={'tmin': 'K', 'tmax': 'K', 'uz': 'm/s',
                 'lat': 'deg', 'ea': 'kpa', 'rs': 'w/m2'}
    ).etr()

# refet eto calc
eto = refet.Daily(
    tmin=pt_data['tasmin'], tmax=pt_data['tasmax'], ea=ea_kpa,
    rs=pt_data['rsds'],
    uz=windspeed, zw=10, elev=round(elev_pt['b1'], 2),
    lat=cell_lat, doy=doy, method='asce',
    input_units={'tmin': 'K', 'tmax': 'K', 'uz': 'm/s',
                 'lat': 'deg', 'ea': 'kpa', 'rs': 'w/m2'}
    ).eto()

# Compare output
print('\nRefET-GEE ETo: {} mm'.format(gee_eto))
print('RefET ETo:     {} mm'.format(float(eto)))
print('\nRefET-GEE ETr: {} mm'.format(gee_etr))
print('RefET ETr:     {} mm'.format(float(etr)))




Maca Class Latitude: 39.14290623931624
Elevation Image Latitude: 39.150000000000006
Input Image Latitude: 39.14290623931624

RefET-GEE ETo: 1.2261928330491418 mm
RefET ETo:     1.2261928402165612 mm

RefET-GEE ETr: 1.8766775404766782 mm
RefET ETr:     1.8766775513488139 mm
